In [ ]:
import random
from classes.minion import Minion
from classes.hero import Hero
from scenarios.deck_1 import deck  # Example deck import
from solver.slv import run_single_turn
from state.transition_state import start_turn, apply_results, end_turn

In [ ]:
from classes.minion import Minion

# Friendly Board (4 minions)
friendly_minions = [
    Minion(
        name="FriendlyMinion A",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=2,
        strat_value=1,
        mana_cost=2
    ),
    Minion(
        name="FriendlyMinion B",
        minion_class="Neutral",
        keywords=[],
        attack=2,
        health=3,
        strat_value=1,
        mana_cost=3
    ),
    Minion(
        name="FriendlyMinion C",
        minion_class="Neutral",
        keywords=[],
        attack=3,
        health=2,
        strat_value=1,
        mana_cost=2
    ),
    Minion(
        name="FriendlyMinion D",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=1,
        strat_value=1,
        mana_cost=1
    )
]

# Enemy Board (3 minions)
enemy_minions = [
    Minion(
        name="EnemyMinion #1",
        minion_class="Neutral",
        # Give it both Taunt=3 and Divine Shield=4
        keywords=[3, 4],
        attack=2,
        health=2,
        strat_value=1,
        mana_cost=1
    ),
    Minion(
        name="EnemyMinion #2",
        minion_class="Neutral",
        keywords=[],
        attack=3,
        health=3,
        strat_value=1,
        mana_cost=3
    ),
    Minion(
        name="EnemyMinion #3",
        minion_class="Neutral",
        keywords=[],
        attack=1,
        health=1,
        strat_value=1,
        mana_cost=1
    )
]

# Hand is empty
hand_list = []


In [ ]:
def main():
    # 1) Create Hero objects (assuming a Hero class exists)
    hero1 = Hero("Paladin", 30)
    hero2 = Hero("Warrior", 30)

    # 2) Print the deck size before drawing
    print(f"Deck has {len(deck)} cards at the start.")

    # 3) Start turn: hero1 draws exactly one card from the deck
    start_turn(hero1, deck, hand_list)

    # Check what got drawn
    print(f"After draw, hand_list has {len(hand_list)} cards.")
    for k, card in enumerate(hand_list):
        print(f"  Hand card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # 4) Build arrays for the solver
    m = len(friendly_minions)   # number of friendly minions on board
    n = len(enemy_minions)      # number of enemy minions on board
    h = len(hand_list)          # number of cards in hand
    M = 5                       # Mana for this turn
    H_hero = 16                 # Opponent hero health

    # ---- Print the counts and minion details ----
    print(f"Number of friendly minions on board: {m}")
    for i, fmn in enumerate(friendly_minions):
        print(f"  - Friendly Minion {i}: {fmn.name} "
              f"(Atk={fmn.attack}, HP={fmn.health})")

    print(f"Number of enemy minions on board: {n}")
    for j, emn in enumerate(enemy_minions):
        print(f"  - Enemy Minion {j}: {emn.name} "
              f"(Atk={emn.attack}, HP={emn.health})")

    print(f"Number of cards in hand: {h}")
    for k, card in enumerate(hand_list):
        print(f"  - Hand Card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # Combine board minions + hand minions for indexing in the solver
    combined = friendly_minions + hand_list
    A = [mn.attack for mn in combined]   # attack values
    B = [mn.health for mn in combined]   # health values
    P = [enm.attack for enm in enemy_minions]
    Q = [enm.health for enm in enemy_minions]

    C = [mn.mana_cost for mn in hand_list]
    S = [mn.strat_value for mn in hand_list]

    # 5) Define or load your weights
    weights = {
        "W1": 12,  # "hero survival penalty" (really means reward for killing hero if used as (1 - z_hero))
        "W2": 10,  # board clear
        "W3": 5,   # face damage
        "W4": 10,  # penalty for leaving enemy minions alive
        "W5": 2,   # friendly minion survival
        "W6": 12,  # reward for minion->minion damage
        "W7": 2,   # mild penalty if your minions take damage
        "W8": 4    # card-play reward
    }



    # 6) Solve (call the new run_single_turn with a weights dict)
    solution = run_single_turn(
        m=m, n=n, h=h,
        M=M, H_hero=H_hero,
        A=A, B=B, P=P, Q=Q, C=C, S=S,
        weights=weights
    )

    # 7) Apply results to scenario_data
    scenario_data = {
        "A": A,
        "B": B,
        "P": P,
        "Q": Q,
        "H_hero": H_hero,
    }
    apply_results(solution, scenario_data)

    # 8) End turn
    end_turn(hero1, hero2)
    print("Turn complete. Inspect logs above for solver outcome.")


if __name__ == "__main__":
    main()